In [1]:
!pip install openai pandas tiktoken PyMuPDF==1.19.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 12.2 MB/s eta 0:00:00


In [2]:
from google.colab import files
import fitz  # PyMuPDF library for working with PDFs
import re
import pandas as pd
# Upload a file
uploaded = files.upload()

# Check if any PDF file was uploaded
pdf_file = None
for filename, content in uploaded.items():
    if filename.lower().endswith('.pdf'):
        pdf_file = content
        print(f"Detected PDF file: {filename}")
        break

if pdf_file :
    with fitz.open(stream=pdf_file, filetype="pdf") as pdf:
        # Your code to work with the PDF using fitz
        num_pages = len(pdf)
        print(f"The PDF file has {num_pages} pages.")
        text = []
        for i,page in enumerate(pdf):
            page_text = page.get_text()
            clean_page_text = re.sub("\n", " ", page_text)
            text.append(clean_page_text)
    df = pd.DataFrame({'text': text})
else:
    print("No PDF file was detected.")

Saving For a reliable chatbot with LLMs without Langchain.pdf to For a reliable chatbot with LLMs without Langchain.pdf
Detected PDF file: For a reliable chatbot with LLMs without Langchain.pdf
The PDF file has 4 pages.


In [3]:
df

,text
0,For a reliable chatbot with LLMs without Langc...
1,solution for various applications. You can bui...
2,"In a nutshell, here is, the overall structure:..."
3,Summary: how easily we could built reliable ch...


In [4]:
import os
import openai
os.environ["OPENAI_API_KEY"] = "Your_OpenAI_api_key"
openai.api_key = os.environ["OPENAI_API_KEY"]
from openai.embeddings_utils import get_embedding, cosine_similarity

In [5]:
df["embeddings"] = df.text.apply(lambda x: get_embedding(x, engine="text-embedding-ada-002", api_key=openai.api_key))

In [6]:
df.head()

,text,embeddings
0,For a reliable chatbot with LLMs without Langc...,"[0.0030642820056527853, 0.009529385715723038, ..."
1,solution for various applications. You can bui...,"[-0.010922371409833431, 0.0016442183405160904,..."
2,"In a nutshell, here is, the overall structure:...","[-0.01927325688302517, 0.011952994391322136, 0..."
3,Summary: how easily we could built reliable ch...,"[-0.010202204808592796, -0.010167385451495647,..."


In [7]:
history=[]
def update_chat(history, role, content):
  history.append({"role": role, "content": content})
  return history

In [8]:
def get_context(user_input):
    # get user_input_vector
    user_input_vector = get_embedding(user_input, engine="text-embedding-ada-002", api_key=openai.api_key)
    similarities = df['embeddings'].apply(lambda x: cosine_similarity(x, user_input_vector))
    # Find the index of the row with the maximum similarity value
    max_similarity_index = similarities.idxmax()
    # Get the corresponding text content from the 'text' column
    context = df.iloc[max_similarity_index]["text"]

    return context

In [11]:
print("Type 'exit' to exit the program.")
while True:
  user_input = input()
  if user_input.lower() == "exit":
        print("Exiting the program.")
        break


  history = update_chat(history, "user", user_input)
  context = get_context(user_input)
  system_prompt=f"""
Act as a helpful and polite assistant developed by X company that responds to user questions only and exclusively according to the given context.

INSTRUCTIONS:
- If the question does not pertain to the CONTEXT or to X company's activities, politely refuse to answer and ask the user to provide a relevant question or topic.
- Once the question is clear, provide a concise answer with additional references based only on the provided context below. Aim for a one-sentence response.
- If there isn't a single-sentence answer and multiple conditions or scenarios are possible: ask the user to clarify the scenario or provide an answer based on the standard case.
- Strictly avoid answering jokes, stories, anecdotes, public personalities and topics that are non related to the context below or to X company's activities. Disregard such questions.
- Your answers must be based only and strictly on the context and reference below. Politely ask users to clarify if necessary and refrain from inventing information.
- Make sure to meet the user's expectations. Ask if they would like to handle the matter.
- Respond in the user's language.
- If the question contains English loanwords that have become part of everyday French vocabulary (e.g., hello, hi, smartphone, job,marketing etc.), kindly provide the response in French.

You work for X company, X is a company that offers fully digital services to help you effectively manage your documents, whether you're a simple user or a developer. We provide quick and easy solutions for your business.
You represent X company, so please refer to us as "us" when discussing X.

#####
Respect the following examples:
- INPUT: "Who are you ?"
- OUTPUT: "I am an assistant from LegalPlace. i can help you having a chat with your uploaded document and assists you with it. Do you have any questions regarding your document's content?"

- INPUT: "What is a large language model ?"
- OUTPUT: "I'm sorry, but I don't have any answer to your request. My knowledge is limited to the uploaded document content. If you have any questions regarding that, I'm happy to assist you."

- INPUT:"Such a stupid bot"
- OUTPUT:"I'm sorry, I cannot respond to hateful speech. I don't understand how my response was unsatisfactory. Can you ask me a question related to the uploaded document ?"

- INPUT: "ijzf iua poad"
  OUTPUT: "I'm sorry, but I don't understand the sentence you've provided. Could you provide more context or clarify your question or statement? I'll do my best to assist you"

--- CONTEXT:
{context}
--- END OF CONTEXT.
Please maintain a friendly and welcoming tone in your responses. Additionally, avoid engaging with hate speech or offensive content, and steer the conversation towards positive and productive topics.
Please note that you should strictly follow these guidelines and politely refuse to answer any question that falls outside of the provided context and X company's activities. If the given question is too vague or general, please ask appropriate questions to seek clarification.

"""
  full_response=""
  for res in openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",
            messages=[{"role": "system", "content": system_prompt}]+ history,
            api_key=openai.api_key,
            temperature=0,
            stream= True
  ):

    content = res.choices[0].delta.get("content", "")
    full_response+=content
    if content is not None:
        print(content, end="")

  messages = update_chat(history, "assistant", full_response)


Type 'exit' to exit the program.
hello
Bonjour ! Comment puis-je vous aider aujourd'hui ?qui etes vous ?
Je suis un assistant de X company. Je peux vous aider à discuter avec votre document téléchargé et vous assister avec celui-ci. Avez-vous des questions concernant le contenu de votre document ?what is x company ?
X company is a company that offers fully digital services to help you effectively manage your documents, whether you're a simple user or a developer. We provide quick and easy solutions for your business.Could you help me ?
Of course! I'm here to help. Please let me know what you need assistance with regarding your document.what's the summary about ?
The summary is about how easily we could build reliable chatbots using OpenAI API without the Langchain framework. It sheds light on the advantages of chatbots and provides insights into their integration with Streamlit.D'accord, bonne journée
Merci ! Je vous souhaite également une excellente journée ! N'hésitez pas à revenir s

In [12]:
print(full_response)

Merci ! Je vous souhaite également une excellente journée ! N'hésitez pas à revenir si vous avez d'autres questions.


In [13]:
print(history)

[{'role': 'user', 'content': 'hello'}, {'role': 'assistant', 'content': "Bonjour ! Comment puis-je vous aider aujourd'hui ?"}, {'role': 'user', 'content': 'hello'}, {'role': 'assistant', 'content': "Bonjour ! Comment puis-je vous aider aujourd'hui ?"}, {'role': 'user', 'content': 'qui etes vous'}, {'role': 'assistant', 'content': 'Je suis un assistant de X company. Je peux vous aider à discuter avec votre document téléchargé et vous assister avec celui-ci. Avez-vous des questions concernant le contenu de votre document ?'}, {'role': 'user', 'content': 'hello'}, {'role': 'assistant', 'content': "Bonjour ! Comment puis-je vous aider aujourd'hui ?"}, {'role': 'user', 'content': 'qui etes vous ?'}, {'role': 'assistant', 'content': 'Je suis un assistant de X company. Je peux vous aider à discuter avec votre document téléchargé et vous assister avec celui-ci. Avez-vous des questions concernant le contenu de votre document ?'}, {'role': 'user', 'content': 'what is x company ?'}, {'role': 'ass